In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_key import geoapify_key

In [19]:
# Load the CSV file
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,10.62,71,75,8.23,CL,1675642456
1,1,karratha,-20.7377,116.8463,32.70,48,3,0.95,AU,1675642872
2,2,rikitea,-23.1203,-134.9692,26.13,75,3,6.69,PF,1675642799
3,3,hermanus,-34.4187,19.2345,21.00,88,77,5.18,ZA,1675642872
4,4,port elizabeth,-33.9180,25.5701,24.50,93,75,4.63,ZA,1675642824


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
# Configure map
humidity_map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo= True,
    tiles= 'EsriNatGeo',
    size= 'Humidity',
    color= 'City',
    alpha= 0.5
)

humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

---

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
# My ideal weather conditions, Max temp = <=110 >=80, Humidity >= 70, Wind Speed <= 14mph,
# any cloudiness, any country
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp'] >=21) & (city_data_df['Max Temp'] <=27) & \
    (city_data_df['Wind Speed'] <=4.5), :]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna(how='any')

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,antofagasta,-23.6500,-70.4000,22.60,87,6,2.74,CL,1675642588
12,12,alofi,-19.0595,-169.9187,22.94,100,91,1.54,NU,1675642877
18,18,georgetown,5.4112,100.3354,25.53,99,20,1.03,MY,1675642879
20,20,kidal,18.4411,1.4078,22.58,12,1,2.35,ML,1675642880
22,22,camopi,3.1655,-52.3158,22.03,99,100,1.71,GF,1675642881


In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,antofagasta,CL,-23.6500,-70.4000,87,
12,alofi,NU,-19.0595,-169.9187,100,
18,georgetown,MY,5.4112,100.3354,99,
20,kidal,ML,18.4411,1.4078,12,
22,camopi,GF,3.1655,-52.3158,99,
...,...,...,...,...,...,...
529,tabuk,PH,17.4189,121.4443,83,
536,beira,MZ,-19.8436,34.8389,85,
544,omboue,GA,-1.5746,9.2618,85,
546,jambi,ID,-1.6000,103.6167,96,


---

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
categories = 'accommodation.hotel'
radius = 10000
limit = 1

params = {
    'categories': categories,
    'radius': radius,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
antofagasta - nearest hotel: Hotel Colón
alofi - nearest hotel: Matavai Resort
georgetown - nearest hotel: Page 63 hostel
kidal - nearest hotel: No hotel found
camopi - nearest hotel: Chez Lucia
hilo - nearest hotel: Dolphin Bay Hotel
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
kabo - nearest hotel: No hotel found
ambilobe - nearest hotel: Hôtel National
banyuwangi - nearest hotel: OYO 1579 Kate'el Inn
merauke - nearest hotel: Coreinn
salinopolis - nearest hotel: Hotel Salinópolis
luanda - nearest hotel: Hotel Forum
qandala - nearest hotel: No hotel found
meulaboh - nearest hotel: Hotel Meuligou
arraial do cabo - nearest hotel: No hotel found
eirunepe - nearest hotel: No hotel found
robertsport - nearest hotel: Nana's Lodge
thongwa - nearest hotel: No hotel found
mapiripan - nearest hotel: No hotel found
mahebourg - nearest hotel: Grand Bel Air
jetafe - nearest hotel: No hotel found
traralgon - nearest hotel: Crown Hotel
cayenne - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
11,antofagasta,CL,-23.6500,-70.4000,87,Hotel Colón
12,alofi,NU,-19.0595,-169.9187,100,Matavai Resort
18,georgetown,MY,5.4112,100.3354,99,Page 63 hostel
20,kidal,ML,18.4411,1.4078,12,No hotel found
22,camopi,GF,3.1655,-52.3158,99,Chez Lucia
...,...,...,...,...,...,...
529,tabuk,PH,17.4189,121.4443,83,Davidson Hotel
536,beira,MZ,-19.8436,34.8389,85,Hotel Savoy
544,omboue,GA,-1.5746,9.2618,85,Georges Faustin Aurat
546,jambi,ID,-1.6000,103.6167,96,Hotel Odua Weston Jambi
